### ~ 15 mins

In [122]:
#default_exp skullstrip

### read -> skull strip -> save

In [1]:
#export
from fastai2.medical.imaging_roi import *
from fastai2.medical.imaging import dicom_windows
from fastai2 import *
from fastai2.torch_core import *
from fastai2.core import *
from fastai2.basics import *

In [29]:
from local.data_prep import _plot_voxel

In [58]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [60]:
# Patient Position Attribute - https://dicom.innolitics.com/ciods/raw-data/general-series/00185100
# We don't normalize it by patient position and leave it as original
# This in a way acts as an inherent data augmentation

### skull strip images

In [89]:
output_path = Path("/home/turgutluk/data/ventricles_data/")

In [96]:
#export 
def _skull_strip_and_save(o):
    "skull strips image using brain mask and saves new image"
    p = o.parent
    suid = o.name.split('_')[0]
    image = torch.load(p/f"{suid}_image.pt")
    brain_mask = torch.load(p/f"{suid}_brain_mask.pt")   
    skull_stripped_image = image*brain_mask
    torch.save(skull_stripped_image, p/f"{suid}_skull_stripped_image.pt")

In [97]:
files = get_files(output_path, extensions=['.pt'])

In [99]:
# 15 mins for all data
parallel(_skull_strip_and_save, files, n_workers=defaults.cpus//2)

(#7504) [None,None,None,None,None,None,None,None,None,None...]

In [103]:
files = get_files(output_path, extensions=['.pt'])

In [106]:
files

(#10004) [/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484791583.866.8004_brain_mask.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.486778702.402.1956_brain_mask.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484728780.561.7428_image.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.486780005.568.3239_ventricles_mask.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11785856.21988995044.507668442.154.6402_ventricles_mask.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.486780130.437.3363_ventricles_mask.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484813575.357.1955_brain_mask.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484726854.615.3528_skull_stripped_image.pt,/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.

In [108]:
o = files[0]

In [114]:
suid = o.name.split("_")[0]
p = o.parent

In [118]:
skull_stripped_image = torch.load(p/f"{suid}_skull_stripped_image.pt")
image = torch.load(p/f"{suid}_image.pt")

In [ ]:
_plot_voxel(skull_stripped_image)

In [ ]:
_plot_voxel(image)

### script

In [ ]:
#export 
import yaml
with open(os.environ.get('YAML_DATA', '../data.yaml')) as f: io = yaml.load(f.read(), yaml.FullLoader)

output_paths = types.SimpleNamespace(
    ATLAS=io['output']['ATLAS'],
    MR=io['output']['MR'],
    CT=io['output']['CT'])

In [ ]:
#export 
from time import perf_counter
@call_parse
def main(output_path:Param("Directory that have data prep results", str)):
    "Read tensors, skull strip images and save it as a new image"
    start = perf_counter()
    
    output_path = Path(output_paths.__dict__[output_path])
    files = get_files(output_path, extensions=['.pt'])
    parallel(_skull_strip_and_save, files, n_workers=defaults.cpus//2)
    
    end = perf_counter()
    print(f"Total time taken {end-start} seconds")

### Export

In [1]:
from local.notebook.export import notebook2script
notebook2script("1b) skull strip.ipynb")

Converted 1b) skull strip.ipynb.
